<a href="https://colab.research.google.com/github/sujit-j/Deep-Learning/blob/main/SpamFilterForQuoraQuestions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2021-07-13 08:36:54--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2021-07-13 08:36:54--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbea57a6d5fc657c1439a92a7ab.dl.dropboxusercontent.com/zip_download_get/A1nmoCEvtbuO4cHJwyE5N_cqQehs1UbBqoT_PRH9cV9Y7rOVtmO_MOQoSv9LQ87cf9vYdN5VBtX9Thd7BzPfZvStqqh0GoGdi3rVdAjJSW1B1g# [following]
--2021-07-13 08:36:54--  https://ucbea57a6d5fc657c1439a92a7ab.dl.dropboxusercontent.com/zip_download_get/A1nmoCEvtbuO4cHJwyE5N_cqQehs1UbBqoT_PRH9cV9Y7rOVtmO_MOQoSv9LQ87cf9vYdN5VBtX9Thd7

In [3]:
!unzip /content/AAAw1_JIzpuVvwteJCma0xMla?dl=0

Archive:  /content/AAAw1_JIzpuVvwteJCma0xMla?dl=0
mapname:  conversion of  failed
 extracting: train.csv               

1 archive had fatal errors.


In [4]:
data = pd.read_csv(r"/content/train.csv")

In [5]:
data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [6]:
data.drop(["qid"],axis=1,inplace=True)

In [7]:
data.head()

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


In [8]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2021-07-13 08:37:26--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2021-07-13 08:37:26--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2021-07-13 08:37:26--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [9]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [10]:
!rm /content/glove.42B.300d.zip

In [11]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [12]:
from sklearn.model_selection import train_test_split
data_train,data_test=train_test_split(data,test_size=0.2)

In [13]:
x_train=data_train['question_text']
y_train=data_train['target']
x_test=data_test['question_text']
y_test=data_test['target']

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM,Activation,Dense,Input,Embedding,Dropout
from tensorflow.keras.models import Model

In [15]:
from nltk import word_tokenize

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
sent_lens=[len(word_tokenize(x)) for x in x_train]

In [18]:
np.percentile(sent_lens,95)

30.0

In [19]:
max_len=30

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [20]:
embedding_matrix=np.zeros((vocab_size+1,300))
words_not_available=0
for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector
    else:
      words_not_available+=1

In [21]:
words_not_available

62232

In [22]:
vocab_size

196379

In [23]:
inputs=Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer=LSTM(50)(embed)
dense1=Dense(10,activation='relu')(lstm_layer)
drop=Dropout(0.2)(dense1)
final_layer=Dense(1,activation='sigmoid')(drop)

model=Model(inputs=inputs,outputs=final_layer)

In [24]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 30)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 300)           58914000  
_________________________________________________________________
lstm (LSTM)                  (None, 50)                70200     
_________________________________________________________________
dense (Dense)                (None, 10)                510       
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 58,984,721
Trainable params: 70,721
Non-trainable params: 58,914,000
____________________________________________

In [25]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [26]:
from sklearn.utils import class_weight
cw = class_weight.compute_class_weight('balanced', 
                                                 np.unique(y_train), 
                                                    y_train)

In [27]:
class_weight_dict = dict(enumerate(cw))
class_weight_dict

{0: 0.5331221383315357, 1: 8.047821867586801}

In [28]:
model.fit(seq_train_matrix,y_train.values,validation_data=(seq_test_matrix,y_test.values),epochs=40,class_weight=class_weight_dict,
          batch_size=30000)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/40
35/35 [==============================] - 328s 9s/step - loss: 0.5187 - accuracy: 0.6980 - val_loss: 0.3849 - val_accuracy: 0.8339
Epoch 2/40
35/35 [==============================] - 326s 9s/step - loss: 0.3426 - accuracy: 0.8564 - val_loss: 0.3189 - val_accuracy: 0.8652
Epoch 3/40
35/35 [==============================] - 322s 9s/step - loss: 0.3127 - accuracy: 0.8711 - val_loss: 0.2946 - val_accuracy: 0.8767
Epoch 4/40
35/35 [==============================] - 328s 9s/step - loss: 0.3001 - accuracy: 0.8744 - val_loss: 0.2917 - val_accuracy: 0.8767
Epoch 5/40
35/35 [==============================] - 329s 9s/step - loss: 0.2922 - accuracy: 0.8773 - val_loss: 0.3134 - val_accuracy: 0.8650
Epoch 6/40
35/35 [==============================] - 337s 10s/step - loss: 0.2867 - accuracy: 0.8781 - val_loss: 0.3098 - val_accuracy: 0.8668
Epoch 7/40
35/3

In [29]:
p=model.predict(seq_test_matrix)

In [30]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,p)

0.9588367266098867